---

# CSCI 3202, Fall 2022
# Homework 2: MDP & Reinforcement Learning
# Due: Friday September 9, 2022 at 6:00 PM

<br> 

### Your name:

<br> 

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict

# added packages
import heapq
from matplotlib import colors
import random
import builtins



---

Consider a **cube** state space defined by $0 \le x, y, z \le L$. Suppose you are piloting/programming a drone to learn how to land on a platform at the center of the $z=0$ surface (the bottom). Some assumptions:
* In this discrete world, if the drone is at $(x,y,z)$ it means that the box is centered at $(x,y,z)$. There are boxes (states) centered at $(x,y,z)$ for all $0 \le x,y,z \le L$. Each state is a 1 unit cube. 
* In this world, $L$ is always an even value.
* All of the states with $z=0$ are terminal states.
* The state at the center of the bottom of the cubic state space is the landing pad. For example, when $L=4$, the landing pad is at $(x,y,z) = (2,2,0)$.
* All terminal states ***except*** the landing pad have a reward of -1. The landing pad has a reward of +1.
* All non-terminal states have a living reward of -0.01.
* The drone takes up exactly 1 cubic unit, and begins in a random non-terminal state.
* The available actions in non-terminal states include moving exactly 1 unit Up (+z), Down (-z), North (+y), South (-y), East (+x) or West (-x). In a terminal state, the training episode should end.

#### Part A
How many states would be in the discrete state space if $L=2$? Explain your reasoning.

*Your answer here:*

`The state space is depending on the volume of the cube. In this case we would have a 27 cubic volume for the state space. My reason for that is because we will have a layer 0, 1, and 2. Which means out state space is more like a rubics cube in where out drone is 1 unit space.`

#### Part B
Write a class `MDPLanding` to represent the Markov decision process for this drone. Include methods for:
1. `actions(state)`, which should return a list of all actions available from the given state
2. `reward(state)`, which should return the reward for the given state
3. `result(state, action)`, which should return the resulting state of doing the given action in the given state

and attributes for:
1. `states`, a list of all the states in the state space, where each state is represented as an $(x,y,z)$ tuple
2. `terminal_states`, a dictionary where keys are the terminal state tuples and the values are the rewards associated with those terminal states
3. `default_reward`, a scalar for the reward associated with non-terminal states
4. `all_actions`, a list of all possible actions (Up, Down, North, South, East, West)
5. `discount`, the discount factor (use $\gamma = 0.999$ for this entire problem)

How you feed arguments/information into the class constructor is up to you.

Note that actions are *deterministic* here.  The drone does not need to include transition probabilities for outcomes of particular actions. What the drone does need to learn, however, is where the landing pad is, and how to get there from any initial state.

Before moving on to Part C, we recommend that you test that your MDPLanding code is set up correctly. Write unit tests that display the actions for a given state, rewards, results, etc. This will help you identify errors in your implementation and save you a lot of debugging time later.

In [1]:
# Solution:

class MDPLanding:
    # attributes
    def __init__(self, state_lst, dict_lst, r):
        # a list of all states in the state space (x,y,z)
        self.states = state_lst 
        # key = terminal state, value = reward of terminal state
        self.terminal_states = dict_lst
        # reward associated with non-terminal state
        self.default_reward = r 
        # list of possible actions
        # self.all_actions = ['Up', 'Down', 'North', 'South', 'East', 'West']
        self.all_actions = [(0,0,1), (0,0,-1), (0,1,0), (0,-1,0), (1, 0, 0), (-1, 0, 0)]
        # discount factor which stays consistance
        self.discount = 0.999

    # return a list of all action available from the given state 
    def actions(self, state):
        if(state[2] == 0):
            result_state = []
            for x in self.all_actions:
                if(x != (0,0,-1)):
                    result_state.append(x)
            return result_state
        
        elif(state[2] == 4):
            result_state = []
            for x in self.all_actions:
                if(x != (0,0,1)):
                    result_state.append(x)
            return result_state

        elif(state[1] == 4):
            result_state = []
            for x in self.all_actions:
                if(x != (0,1,0)):
                    result_state.append(x)
            return result_state             

        elif(state[1] == 0):
            result_state = []
            for x in self.all_actions:
                if(x != (0,-1,0)):
                    result_state.append(x)
            return result_state             

        elif(state[0] == 4):
            result_state = []
            for x in self.all_actions:
                if(x != (1,0,0)):
                    result_state.append(x)
            return result_state            

        elif(state[0] == 0):
            result_state = []
            for x in self.all_actions:
                if(x != (-1,0,0)):
                    result_state.append(x)
            return result_state         

        else:
            return self.all_actions
            
    #  return the reward for the given state
    def reward(self, state):
        if(state in self.terminal_states):
            return self.terminal_states[state] 
        else:
            return self.default_reward
            
    # return the resulting state of doing the given action in the given state
    def result(self, state, action):
        res = tuple(map(lambda i, j: i + j, state, action))
        # if statments are used to check if you would hit an unaccecable location by taking that actions
        if(res[0] > 4 or res[0] < 0):
            return state
        elif(res[1] > 4 or res[1] < 0):
            return state
        elif(res[2] > 4 or res[2] < 0):
            return state
        else:
            return res
    

#### Part C
Write a function to implement **policy iteration** for this drone landing MDP. Create an MDP environment to represent the $L=4$ case.

Use your function to find an optimal policy for your new MDP environment. Check (by printing to screen) that the policy for the following states are what you expect, and **comment on the results**:
1. $(2,2,1)$
1. $(0,2,1)$
1. $(2,0,1)$

The policy for each of these states is the action that the agent should take in that state. 

In [2]:
# State from l = 4
staa = []
x=0
y=0
z=0

for x in range(5):
    for y in range(5):
        for z in range(5):
            staa.append((x, y, z))

result = list(
    filter(
        lambda tup: tup[2] == 0,
        staa
    )
)

# rewards and punishment for terminal state spaces 
my_dict = {}
for i in result:
    if(i == (2,2,0)):
        my_dict[i] = 1.0
    else:
        my_dict[i] = -1.0
        
print(my_dict)

{(0, 0, 0): -1.0, (0, 1, 0): -1.0, (0, 2, 0): -1.0, (0, 3, 0): -1.0, (0, 4, 0): -1.0, (1, 0, 0): -1.0, (1, 1, 0): -1.0, (1, 2, 0): -1.0, (1, 3, 0): -1.0, (1, 4, 0): -1.0, (2, 0, 0): -1.0, (2, 1, 0): -1.0, (2, 2, 0): 1.0, (2, 3, 0): -1.0, (2, 4, 0): -1.0, (3, 0, 0): -1.0, (3, 1, 0): -1.0, (3, 2, 0): -1.0, (3, 3, 0): -1.0, (3, 4, 0): -1.0, (4, 0, 0): -1.0, (4, 1, 0): -1.0, (4, 2, 0): -1.0, (4, 3, 0): -1.0, (4, 4, 0): -1.0}


In [3]:
r = -0.01
mpd = MDPLanding(staa, my_dict, r)

def expected_utility(a, s, U, mdp):
    prob = 1
    next_move = mdp.result(s, a)
    utility = prob * U[next_move]
    return utility

def policy_evaluation(pi, U, mdp):
    gama = mdp.default_reward
    for s in mdp.states:
        R = mdp.reward(s)
        U[s] = R + gama + (expected_utility(pi[s], s, U, mdp))
    return U

def max_unility_helper(a, s, u, mpd):
    all_utility_from_action = {}
    for i in range(len(a)):
        curr_u = expected_utility(a[i], s, u, mpd)
        all_utility_from_action[a[i]]=(curr_u)
        
    max_key = max(all_utility_from_action, key=all_utility_from_action.get)
    return max_key, all_utility_from_action[max_key]
    
def policy_iteration(mdp):
    U = {s: mdp.default_reward for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        unchanged = True
        for s in mdp.states:
            t = mdp.actions(s)
            a, tt = max_unility_helper(t, s, U, mdp)
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            return pi, U
    

In [6]:
best_policy, best_utility = policy_iteration(mpd)

print("For move (0,2,1) the best move was: ",best_policy[(0,2,1)], "should be East")
print()
print("For move (2,2,1) the best move was: ",best_policy[(2,2,1)], "should be Down")
print()
print("For move (2,0,1) the best move was: ",best_policy[(2,0,1)], "should be North")

For move (0,2,1) the best move was:  (1, 0, 0) should be East

For move (2,2,1) the best move was:  (0, 0, -1) should be Down

For move (2,0,1) the best move was:  (0, 1, 0) should be North


In [7]:
print(best_utility)

{(0, 0, 0): -1.1400000000000001, (0, 0, 1): 0.8399999999999999, (0, 0, 2): 0.8199999999999998, (0, 0, 3): 0.7999999999999998, (0, 0, 4): 0.7799999999999998, (0, 1, 0): -0.15000000000000013, (0, 1, 1): 1.8299999999999998, (0, 1, 2): 1.8099999999999998, (0, 1, 3): 1.7899999999999998, (0, 1, 4): 1.7699999999999998, (0, 2, 0): 0.8599999999999999, (0, 2, 1): 2.82, (0, 2, 2): 2.8, (0, 2, 3): 2.78, (0, 2, 4): 2.76, (0, 3, 0): 0.8199999999999998, (0, 3, 1): 2.8, (0, 3, 2): 2.78, (0, 3, 3): 2.76, (0, 3, 4): 2.7399999999999998, (0, 4, 0): 0.7999999999999998, (0, 4, 1): 2.78, (0, 4, 2): 2.76, (0, 4, 3): 2.7399999999999998, (0, 4, 4): 2.7199999999999998, (1, 0, 0): -0.15000000000000013, (1, 0, 1): 1.8299999999999998, (1, 0, 2): 1.8099999999999998, (1, 0, 3): 1.7899999999999998, (1, 0, 4): 1.7699999999999998, (1, 1, 0): 0.8599999999999999, (1, 1, 1): 2.82, (1, 1, 2): 2.8, (1, 1, 3): 2.78, (1, 1, 4): 2.76, (1, 2, 0): 2.84, (1, 2, 1): 3.8099999999999996, (1, 2, 2): 3.7899999999999996, (1, 2, 3): 3.76

#### Part D
Provide an example of a non-deterministic transition that could be included in your code in Part C. Describe the function. How would you modify your code to handle a non-deterministic transition function?

*Your answer here*

`Our drone flys in a non-deterministic motion, so there is not probability involed with its movements. The way our code ensure this is with the calculation of Expected utility. Normally we would sum the probability of choose our next action but because there is already a chossen next action, we dont need to include the probabily asspect. `

#### Part E
Describe the main differences between **policy iteration** and **value iteration**? How would your code change in Part C to convert it to **value iteration**?

*Your answer here:*

`Value iteration keeps track of utilities and will terminate when updates are at some thresh hold. Policy iteration also stores policy and will terminate when the policy terminates. In our code we would have to convert our calculation to store the next move. We would insted of chooseing the next utility based on random action through out the board, we will run through every possible action avaliable to us in that given state and apply a greedy algorithm to store the best possible action to choose. `

#### Part F

Code up a **Q-learning** agent/algorithm to learn how to land the drone. You can do this however you like, as long as you use the MDP class structure defined above.  

Your code should include some kind of a wrapper to run many trials to train the agent and learn the Q values.  You also do not need to have a separate function for the actual "agent"; your code can just be a "for" loop within which you are refining your estimate of the Q values.

From each training trial, save the cumulative discounted reward (utility) over the course of that episode. That is, add up all of $\gamma^t R(s_t)$ where the drone is in state $s_t$ during time step $t$, for the entire sequence. We refer to this as "cumulative reward" because we usually refer to "utility" as the utility *under an optimal policy*.

Some guidelines:
* The drone should initialize in a random non-terminal state for each new training episode.
* The training episodes should be limited to 50 time steps, even if the drone has not yet landed. If the drone lands (in a terminal state), the training episode is over.
* You may use whatever learning rate $\alpha$ you decide is appropriate, and gives good results.
* There are many forms of Q-learning. You can use whatever you would like, subject to the reliability targets below.
* Your code should return:
  * The learned Q values associated with each state-action pair.
  * The cumulative reward for each training trial. 
  * Anything else that might be useful in the ensuing analysis.

In [8]:
# Solution:
results = [t for t in mpd.states if t[2] > 0]
print(results)

def temporal_difference(s,a, mdp, u, poli, iteration):
    t = mdp.result(s,a)
    list_ac = mdp.actions(s)
    max_t, tt_u = max_unility_helper(list_ac, s, u, mpd)
    at = 60/59+iteration
    r = mdp.reward(t)
    adjust = tt_u - u[t]
    temporal = at * (r + mdp.discount*(adjust))
    return max_t, temporal
    
def Q_learning(mdp, inital_policy):
    Q = {}
    ittter = 50
    for i in range(200):
        new_start = random.choice(results)
        print(new_start)
        for x in range(50):
            if (new_start in mpd.terminal_states):
                return Q
            else:
                # pick an action using greedy 
                next_act = inital_policy[new_start]
                # count stat-action pair 
                
                # calculate Q[s,a] for action selected
                tempo_p, tempo_u = temporal_difference(new_start, next_act, mdp, best_utility, inital_policy,ittter)
                
                # update policy for action at Q[s,a]
                inital_policy[new_start] = tempo_p
                
                #calculat commulative rewards using discound, step, rcs(reward matrix)
                tempo_u
        Q[new_start] = tempo_u
        ittter = ittter+1
    return Q

[(0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 2, 4), (0, 3, 1), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 4), (1, 0, 1), (1, 0, 2), (1, 0, 3), (1, 0, 4), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 2, 4), (1, 3, 1), (1, 3, 2), (1, 3, 3), (1, 3, 4), (1, 4, 1), (1, 4, 2), (1, 4, 3), (1, 4, 4), (2, 0, 1), (2, 0, 2), (2, 0, 3), (2, 0, 4), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 3, 1), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 4, 1), (2, 4, 2), (2, 4, 3), (2, 4, 4), (3, 0, 1), (3, 0, 2), (3, 0, 3), (3, 0, 4), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 1, 4), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 2, 4), (3, 3, 1), (3, 3, 2), (3, 3, 3), (3, 3, 4), (3, 4, 1), (3, 4, 2), (3, 4, 3), (3, 4, 4), (4, 0, 1), (4, 0, 2), (4, 0, 3), (4, 0, 4), (4, 1, 1), (4, 1, 2), (4, 1, 3), (4, 1, 4), (4, 2, 1), (4, 2, 2), (4, 2, 3)

In [9]:
Q_learning(mpd, best_policy)

(4, 1, 2)
(2, 2, 1)
(4, 3, 1)
(0, 2, 1)
(3, 3, 1)
(1, 2, 3)
(2, 4, 3)
(3, 4, 1)
(1, 4, 4)
(3, 4, 4)
(1, 1, 4)
(4, 0, 1)
(2, 1, 3)
(0, 1, 4)
(4, 1, 4)
(0, 2, 4)
(4, 0, 4)
(1, 3, 2)
(4, 1, 1)
(0, 1, 2)
(4, 2, 1)
(0, 3, 4)
(4, 4, 4)
(2, 0, 4)
(4, 1, 4)
(2, 3, 4)
(2, 4, 4)
(2, 2, 2)
(1, 1, 3)
(1, 0, 3)
(0, 4, 1)
(0, 3, 1)
(4, 0, 1)
(0, 3, 1)
(0, 1, 3)
(3, 1, 2)
(0, 3, 2)
(2, 3, 3)
(1, 1, 3)
(2, 3, 2)
(0, 1, 4)
(2, 2, 2)
(4, 0, 4)
(0, 4, 4)
(3, 4, 1)
(1, 1, 4)
(0, 0, 4)
(3, 4, 2)
(1, 2, 4)
(2, 4, 2)
(1, 0, 3)
(1, 0, 1)
(4, 4, 1)
(0, 1, 3)
(0, 1, 3)
(2, 4, 4)
(0, 1, 4)
(2, 0, 4)
(2, 3, 2)
(2, 4, 2)
(4, 4, 4)
(0, 1, 2)
(2, 3, 2)
(0, 0, 3)
(0, 1, 2)
(4, 1, 1)
(0, 0, 1)
(4, 2, 1)
(4, 4, 4)
(0, 3, 1)
(1, 0, 1)
(3, 0, 4)
(4, 1, 4)
(3, 2, 2)
(2, 0, 2)
(3, 2, 3)
(3, 0, 4)
(1, 0, 1)
(2, 0, 2)
(3, 3, 4)
(3, 1, 1)
(3, 1, 3)
(3, 0, 2)
(0, 2, 1)
(4, 1, 1)
(4, 4, 1)
(1, 1, 4)
(2, 1, 3)
(1, 3, 1)
(3, 4, 2)
(3, 0, 2)
(2, 3, 4)
(0, 4, 4)
(4, 1, 4)
(0, 2, 2)
(3, 0, 3)
(4, 2, 4)
(2, 1, 4)
(0, 1, 2)
(0, 3, 4)


{(4, 1, 2): -2.2001694915254237,
 (2, 2, 1): 162.01694915254237,
 (4, 3, 1): -2.4601694915254235,
 (0, 2, 1): -1.3401694915254236,
 (3, 3, 1): -1.9901694915254238,
 (1, 2, 3): -0.5601694915254237,
 (2, 4, 3): -0.5701694915254237,
 (3, 4, 1): -2.1801694915254237,
 (1, 4, 4): -1.6801694915254237,
 (3, 4, 4): -1.9701694915254238,
 (1, 1, 4): -2.420169491525424,
 (4, 0, 1): -1.8701694915254237,
 (2, 1, 3): -2.3501694915254236,
 (0, 1, 4): -1.0701694915254236,
 (4, 1, 4): -1.4401694915254237,
 (0, 2, 4): -2.4901694915254238,
 (4, 0, 4): -2.1001694915254236,
 (1, 3, 2): -2.4501694915254237,
 (4, 1, 1): -2.380169491525424,
 (0, 1, 2): -2.4701694915254238,
 (4, 2, 1): -1.1801694915254237,
 (0, 3, 4): -1.8201694915254236,
 (4, 4, 4): -2.280169491525424,
 (2, 0, 4): -1.0801694915254236,
 (2, 3, 4): -2.050169491525424,
 (2, 4, 4): -1.0601694915254236,
 (2, 2, 2): -1.9201694915254237,
 (1, 1, 3): -0.8901694915254237,
 (1, 0, 3): -2.4401694915254235,
 (0, 4, 1): -2.070169491525424,
 (0, 3, 1): -1.5

#### Part G

Initialize the $L=10$ environment (so that the landing pad is at $(5,5,0)$). Run some number of training trials to train the drone.

**How do I know if my drone is learned enough?**  If you take the mean cumulative reward across the last 5000 training trials, it should be around 0.80. This means at least about 10,000 (but probably more) training episodes will be necessary. It will take a few seconds on your computer, so start small to test your code.

**Then:** Compute block means of cumulative reward from all of your training trials. Use blocks of 500 training trials. This means you need to create some kind of array-like structure such that its first element is the mean of the first 500 trials' cumulative rewards; its second element is the mean of the 501-1000th trials' cumulative rewards; and so on. Make a plot of the block mean rewards as the training progresses. It should increase from about -0.5 initially to somewhere around +0.8.

**And:** Print to the screen the mean of the last 5000 trials' cumulative rewards, to verify that it is indeed about 0.80.

In [10]:
# Solution:

# State from l = 10
staa = []
x=0
y=0
z=0

for x in range(11):
    for y in range(11):
        for z in range(11):
            staa.append((x, y, z))

result = list(
    filter(
        lambda tup: tup[2] == 0,
        staa
    )
)

# rewards and punishment for terminal state spaces 
my_dict = {}
for i in result:
    if(i == (5,5,0)):
        my_dict[i] = 1.0
    else:
        my_dict[i] = -1.0
        
        
print("L=10", staa)
print("Therminal state rewards:", my_dict)

L=10 [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 0, 5), (0, 0, 6), (0, 0, 7), (0, 0, 8), (0, 0, 9), (0, 0, 10), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 1, 10), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 2, 10), (0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 3, 10), (0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 4), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 4, 10), (0, 5, 0), (0, 5, 1), (0, 5, 2), (0, 5, 3), (0, 5, 4), (0, 5, 5), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 5, 10), (0, 6, 0), (0, 6, 1), (0, 6, 2), (0, 6, 3), (0, 6, 4), (0, 6, 5), (0, 6, 6), (0, 6, 7), (0, 6, 8), (0, 6, 9), (0, 6, 10), (0, 7, 0), (0, 7, 1), (0, 7, 2), (0, 7, 3), (0, 7, 4), (0, 7, 5), (0, 7, 6), (0, 7, 7), (0, 7, 8), (0, 7, 9), (0, 7, 10), (0, 8, 0), (0, 8, 

#### Part H

**Question 1:** Why does the cumulative reward start off around -0.5 at the beginning of the training?

**Question 2:** Why will it be difficult for us to train the drone to reliably obtain rewards much greater than about 0.8?

**Your answer here:**



**Question 1 Answer:** `The comulative rewards always start off at -0.5 because of how the probability if broken down. Because of the use of implementing the greedy approach, we would have our two action of keeping the currently policy or changing it to the new maximum. Leaving it to be the 1-0.5 but because in our situation with the descrete movmenet it would be a negative 0.5. `

**Question 2 Answer:** `Besides the fact of the million of runs the drone would have to preform to get a number close to 0.8, its also because its almost impossible to get it that high. Not only will there be hundreds of other varibles like weather and outside recourses, the thought prosses of constantly updating the policy will cause the reward to be a punishment on a run that might of been the most optimal on that action.`